# Déchiffrement de la syntaxe Prolog

In [2]:
type var = Id of string * int

type table = Empty | NonEmpty of term * table
and term = Var of var | Predicate of string * (term list) | Table of table

type clause = Clause of term * (term list);;

type var = Id of string * int


type table = Empty | NonEmpty of term * table
and term = Var of var | Predicate of string * term list | Table of table


type clause = Clause of term * term list


## Le nouveau Parser :

In [3]:
type structure = S of string | V of var | L of table | T of term | C of clause 
                | W of clause list | P of string | TL of term list 


type structure =
    S of string
  | V of var
  | L of table
  | T of term
  | C of clause
  | W of clause list
  | P of string
  | TL of term list


In [4]:
let to_list str = List.init (String.length str) (String.get str);;
let rec remove_spaces = function
| [] -> []
| (' ' | '\t' | '\n')::l -> remove_spaces l
| a::l -> a::(remove_spaces l);;
let (|>) a b = b a;;
let of_char c = String.make 1 c;;

val to_list : string -> char list = <fun>


val remove_spaces : char list -> char list = <fun>


val ( |> ) : 'a -> ('a -> 'b) -> 'b = <fun>


val of_char : char -> string = <fun>


In [5]:
let ( <+> ) ra rb = fun l -> (ra l) @ (rb l)

let chain y l = List.map (fun (a, b) -> (a, y@b)) l

let ( *~* ) ra rb = 
let chain y l = List.map (fun (a, b) -> (a, y@b)) l in
                    fun l -> List.concat (
                        List.map (fun (x, y) -> chain y (rb x)) (ra l)
                    )
                    
let ( >~> ) ra f = fun l -> List.map (fun (a, b) -> (a, f b)) (ra l)

let skip = fun l -> []
                    
let fix aux = let rec g x = aux g x in g

val ( <+> ) : ('a -> 'b list) -> ('a -> 'b list) -> 'a -> 'b list = <fun>


val chain : 'a list -> ('b * 'a list) list -> ('b * 'a list) list = <fun>


val ( *~* ) :
  ('a -> ('b * 'c list) list) ->
  ('b -> ('d * 'c list) list) -> 'a -> ('d * 'c list) list = <fun>


val ( >~> ) : ('a -> ('b * 'c) list) -> ('c -> 'd) -> 'a -> ('b * 'd) list =
  <fun>


val skip : 'a -> 'b list = <fun>


val fix : (('a -> 'b) -> 'a -> 'b) -> 'a -> 'b = <fun>


In [6]:
let rlow = function
| ('a'..'z' as c)::l -> [l, [S (of_char c)]]
| _ -> []

let rup = function
| ('A'..'Z' as c)::l -> [l, [S (of_char c)]]
| _ -> []

let rsym s = function
| h::t when s = h -> [t, [S (of_char s)]]
| _ -> []

let rchar = function
| ('a'..'z' | 'A'..'Z' | '0'..'9' | '_' as c)::t -> [t, [S (of_char c)]]
| _ -> []

let rmot =
let aux self = rchar 
            <+> rchar *~* self >~> (function | [S s; S s'] -> [S (s^s')] | l -> l)
in fix aux

let variable = 
    rup >~> (function | [S s] -> [V (Id (s,0) )] | l -> l)
<+> rup *~* rmot >~> (function | [S s; S s'] -> [V (Id ((s^s'),0) )] | l -> l)

let predicate = rlow >~> (function | [S s] -> [P s] | l -> l)
                <+> rlow *~* rmot >~> (function | [S s; S s'] -> [P (s^s')] | l -> l)
                
let rec term l = (
    variable >~> (function | [V s] -> [T (Var s)] | l -> l)
<+> predicate >~> (function | [P s] -> [T (Predicate (s,[]))] | l -> l)
<+> predicate *~* (rsym '(' >~> skip) *~* termlist *~* (rsym ')' >~> skip) 
        >~> (function | [P s; TL tl] -> [T (Predicate (s, tl))] | l -> l)
) l
and termlist l = (
term >~> (function | [T t] -> [TL [t]] | l -> l)
<+> term *~* (rsym ',' >~> skip) *~* termlist >~> (function | [T t; TL tl] -> [TL (t::tl)] | l -> l)
) l

let clause = term *~* (rsym '.' >~> skip) >~> (function | [T t] -> [C (Clause (t,[]))] | l -> l)
            <+> term *~* (rsym ':' *~* rsym '-' >~> skip) *~* termlist *~* (rsym '.' >~> skip)
                    >~> (function | [T t; TL tl] -> [C (Clause (t,tl))] | l -> l)
let programme =
let aux self = clause >~> (function | [C c] -> [W [c]] | l -> l)
            <+> clause *~* self >~> (function | [C c; W cl] -> [W (c::cl)]| l -> l)
in fix aux

let check r l = 
let aux = function
| [], result -> Some result
| _ -> None
in match List.filter_map aux (r l) with
| [] -> failwith "Parsing failed"
| [a] -> a
| _ -> failwith "Ambiguous grammar";;

let parse_program s = match (s |> to_list |> remove_spaces |> (check programme)) with
| [W w] -> w
| _ -> failwith "input is not a program"

type var = Id of string * int


type table = Empty | NonEmpty of term * table
and term = Var of var | Predicate of string * term list | Table of table


type clause = Clause of term * term list


type structure =
    S of string
  | V of var
  | L of table
  | T of term
  | C of clause
  | W of clause list
  | P of string
  | TL of term list


val rlow : char list -> (char list * structure list) list = <fun>


...

val term : char list -> (char list * structure list) list = <fun>
val termlist : char list -> (char list * structure list) list = <fun>


val clause : char list -> (char list * structure list) list = <fun>


val programme : char list -> (char list * structure list) list = <fun>


val check : ('a -> ('b list * 'c) list) -> 'a -> 'c = <fun>


val parse_program : string -> clause list = <fun>


# Algorithme d'unification

In [7]:
(* Teste si une variable est dans un terme *)
let rec var_in_term v = function
| Var w -> v = w
| Term (_, l) -> List.mem true (List.map (var_in_term v) l)
| PList Empty -> false
| PList (PLCons (l, t)) -> List.mem true (List.map (var_in_term v) l) || var_in_term v t;;

var_in_term (VarT ("P", 0)) (parser_term "[A,[B,Z],C | [E | P] ]");;

error: compile_error

In [14]:
(* Remplace une variable par un nouveau terme, récursivement à l'intérieur d'un terme *)
let rec replace_var_in_term var new_t = function
| Var v when v = var -> new_t
| Var v -> Var v
| Term (a , l) -> Term (a, List.map (replace_var_in_term var new_t) l)
| PList Empty -> PList Empty
| PList (PLCons (l, t)) -> PList 
(PLCons ((List.map (replace_var_in_term var new_t) l), (replace_var_in_term  var new_t t)));;

replace_var_in_term (VarL ("E",0)) (parser_term "pierre") (parser_term "etudiant_de([E,B | E],P)");;

val replace_var_in_term : var -> term -> term -> term = <fun>


- : term =
Term ("etudiant_de",
 [PList
   (PLCons ([Var (VarT ("E", 0)); Var (VarT ("B", 0))], Term ("pierre", [])));
  Var (VarT ("P", 0))])


In [15]:
(* Teste si une variable est dans une liste de couples de termes *)
let var_in_eql v e = List.mem true (List.map (function t -> var_in_term v (fst t) || var_in_term v (snd t)) e);;

(* Remplace une variable par un nouveau terme, à l'intérieur d'une liste de couples de termes *)
let replace_var_in_eql var new_t eql = 
List.map (function (a, b) -> (replace_var_in_term var new_t a), (replace_var_in_term var new_t b)) eql;;

val var_in_eql : var -> (term * term) list -> bool = <fun>


val replace_var_in_eql :
  var -> term -> (term * term) list -> (term * term) list = <fun>


In [16]:
(* E est une liste de couples de termes *)
(* transforme E en un équivalent sous forme résolue (type option, None si impossible) *) 
let rec solve e =
    (* Parcourt une fois l'ensemble des équations *)
    let rec pass tmp = function
    | [] -> Some tmp
    
    | (Term (f, lf), Term (g, lg))::l when f = g && List.length lf = List.length lg -> 
        pass ((List.map2 (fun a b -> a,b) lf lg)@tmp) l
        
    | (Term (_, _), Term (_, _))::l -> None (* failwith "Unification failed" *)
    
    | (Var x, Var y)::l when x = y -> pass tmp l
    
    | (Term (a, la), Var x)::l -> pass ((Var x, Term (a, la))::tmp) l
    
    | (Var x, t)::l when var_in_eql x tmp || var_in_eql x l -> 
        if var_in_term x t then None (* failwith "Unification failed, loop" *)
        else pass ((Var x, t)::(replace_var_in_eql x t tmp)) (replace_var_in_eql x t l)
        
    | (Var x, t)::l -> pass ((Var x, t)::tmp) l
    
in match (pass [] e) with 
| None -> None
| Some eqlpp -> let b = List.sort_uniq compare eqlpp in if e = b then Some e else solve b;;

File "[16]", line 5, characters 23-712:
 5 | .......................function
 6 |     | [] -> Some tmp
 7 |     
 8 |     | (Term (f, lf), Term (g, lg))::l when f = g && List.length lf = List.length lg -> 
 9 |         pass ((List.map2 (fun a b -> a,b) lf lg)@tmp) l
...
18 |         if var_in_term x t then None (* failwith "Unification failed, loop" *)
19 |         else pass ((Var x, t)::(replace_var_in_eql x t tmp)) (replace_var_in_eql x t l)
20 |         
21 |     | (Var x, t)::l -> pass ((Var x, t)::tmp) l
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Term (_, _), PList _)::_|(PList _, _)::_)


val solve : (term * term) list -> (term * term) list option = <fun>


In [17]:
solve [parser_term "etudiant_de(E, P)", parser_term "etudiant_de(S, P)"];; (* E = S *)

- : (term * term) list option =
Some [(Var (VarT ("E", 0)), Var (VarT ("S", 0)))]


In [18]:
(* Prend 2 terme et renvoie l'unifieur le plus général s'il existe, None sinon. 
Attention, aucune variable ne doit être présente dans (r) la requête et dans (c) la tête de la clause *)

let rec mgu r c = let f = function
    | Var x, Term (a, l) -> x, Term (a, l)
    | Var x, Var y -> x, Var y
    | Term (_, _), _ -> failwith "Should not happen."
in match solve [r, c] with
| None -> None
| Some e -> Some (List.map f e);;

File "[18]", line 4, characters 26-162:
4 | ..........................function
5 |     | Var x, Term (a, l) -> x, Term (a, l)
6 |     | Var x, Var y -> x, Var y
7 |     | Term (_, _), _ -> failwith "Should not happen."
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Var _, PList _)|(PList _, _))


val mgu : term -> term -> (var * term) list option = <fun>


In [19]:
(* La fonction test_unification a pour but de tester mgu *)
let test_unification str_r str_c = mgu (parser_term str_r) (parser_term str_c);;

test_unification "etudiant_de(E, pierre)" "etudiant_de(F,P)";; (* F/E et P/pierre *)

test_unification "etudiant_de(F,P)" "etudiant_de(E, pierre)";; (* E/F et P/pierre *)

test_unification "f(X,g(Y))" "f(g(Z),Z)";; (* X/g(g(Y)) et Z/g(Y) *)

test_unification "f(g(Z),Z)" "f(X,g(Y))";; (* X/g(g(Y)) et Z/g(Y) *)

val test_unification : string -> string -> (var * term) list option = <fun>


- : (var * term) list option =
Some
 [(VarT ("E", 0), Var (VarT ("F", 0))); (VarT ("P", 0), Term ("pierre", []))]


- : (var * term) list option =
Some
 [(VarT ("F", 0), Var (VarT ("E", 0))); (VarT ("P", 0), Term ("pierre", []))]


- : (var * term) list option =
Some
 [(VarT ("X", 0), Term ("g", [Term ("g", [Var (VarT ("Y", 0))])]));
  (VarT ("Z", 0), Term ("g", [Var (VarT ("Y", 0))]))]


- : (var * term) list option =
Some
 [(VarT ("X", 0), Term ("g", [Term ("g", [Var (VarT ("Y", 0))])]));
  (VarT ("Z", 0), Term ("g", [Var (VarT ("Y", 0))]))]


# Algorithme de backtracking

In [20]:
(* Applique une substitution sur un terme *)
let rec apply_subst_on_term uni term = match uni with
| [] -> term
| (v,t)::l -> apply_subst_on_term l (replace_var_in_term v t term);;

(* Applique une substitution sur une liste de termes *)
let rec apply_subst_on_termlist uni lst = List.map (apply_subst_on_term uni) lst;;

val apply_subst_on_term : (var * term) list -> term -> term = <fun>


val apply_subst_on_termlist : (var * term) list -> term list -> term list =
  <fun>


In [21]:
(* Prend un entier et une clause, renvoie la clause avec les variables renommées à l'entier *)
let rename n (Clause (t1, tl)) = 
let rec f = function 
| Var (VarT (str, _)) -> Var (VarT (str, n))
| Term (atm, l) -> Term (atm, List.map f l)
in Clause (f t1, List.map f tl);;

rename 42 (parser_clause "etudiant_de(E,P):-apprend(E,M), enseigne(P,M)");;

File "[21]", line 3, characters 12-110:
3 | ............function 
4 | | Var (VarT (str, _)) -> Var (VarT (str, n))
5 | | Term (atm, l) -> Term (atm, List.map f l)
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
(Var (VarL (_, _))|PList _)


val rename : int -> clause -> clause = <fun>


- : clause =
Clause (Term ("etudiant_de", [Var (VarT ("E", 42)); Var (VarT ("P", 42))]),
 [Term ("apprend", [Var (VarT ("E", 42)); Var (VarT ("M", 42))]);
  Term ("enseigne", [Var (VarT ("P", 42)); Var (VarT ("M", 42))])])


In [22]:
type 'a tree = Leaf of 'a | Node of (('a tree) Lazy.t) list;;

(* La fonction de recherche, renvoie l'arbre des solutions *)
let rec sld_tree world req subs n = match req with
| [] -> Leaf subs 
| head_request_term::other_request_terms -> 
    Node (List.filter_map (fun c -> let Clause (left_member, right_member) = rename n c in 
    (match mgu head_request_term left_member with
        | None -> None
        | Some unifier -> Some (lazy (sld_tree world 
            (apply_subst_on_termlist unifier (right_member@other_request_terms)) 
            (unifier::subs) (n+1)))
    )) world);;

type 'a tree = Leaf of 'a | Node of 'a tree Lazy.t list


val sld_tree :
  clause list ->
  term list -> (var * term) list list -> int -> (var * term) list list tree =
  <fun>


In [23]:
(* Compose correctement la liste de substitutions *)
let compose l = 
let compose2 s1 s2 = s1@(List.map (function (v2, lt2) -> (v2, apply_subst_on_term s1 lt2)) s2)
in List.fold_left compose2 [] l;;

(* Recherche les termes qui sont des variables récursivement dans une liste de termes *)
let rec find_vars_in_termlist tl = List.sort_uniq compare (List.concat (List.map 
(function 
| Var v -> [Var v]
| Term (_,l) -> find_vars_in_termlist l) 
tl));;

val compose : (var * term) list list -> (var * term) list = <fun>


File "[23]", line 8, characters 0-70:
 8 | (function 
 9 | | Var v -> [Var v]
10 | | Term (_,l) -> find_vars_in_termlist l).
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
PList _


val find_vars_in_termlist : term list -> term list = <fun>


In [24]:
let list_to_seq l = List.fold_right (fun x s -> (fun ()->Seq.Cons(x, s)) ) l Seq.empty;;

let rec to_seq = function
| Leaf str -> Seq.return str
| Node tl -> Seq.flat_map (fun par -> to_seq (Lazy.force par)) (list_to_seq tl);;

let solutions world req = let tree = sld_tree world (parser_term_list req) [] 1 in 
let vars = (find_vars_in_termlist (parser_term_list req)) in
Seq.map  (fun l -> vars, apply_subst_on_termlist (compose l) vars) (to_seq tree);;

let request world req = let sol = solutions world req in
if sol () = Seq.Nil then Format.printf "This is false.\n%!"
else Seq.iter (fun (vars, tl) -> 
if vars = [] then Format.printf "This is True.\n%!"
else Format.printf "There is : %s\n%!"
(String.concat ", " (List.map2 (fun v t -> (term_to_string v) ^ " = " ^ (term_to_string t)) vars tl))
) sol;;

val list_to_seq : 'a list -> 'a Seq.t = <fun>


val to_seq : 'a tree -> 'a Seq.t = <fun>


val solutions : clause list -> string -> (term list * term list) Seq.t =
  <fun>


val request : clause list -> string -> unit = <fun>


# Essais de l'implémentation

## Tests basiques... des élèves, des cours et des professeurs

In [25]:
let program = "apprend(eve, mathematiques).
apprend(benjamin, informatique).
apprend(benjamin, physique).
enseigne(alice, physique).
enseigne(pierre, mathematiques).
enseigne(pierre, informatique).
etudiant_de(E,P):-apprend(E,M), enseigne(P,M).
";;

lexer program;;

val program : string =
  "apprend(eve, mathematiques).\napprend(benjamin, informatique).\napprend(benjamin, physique).\nenseigne(alice, physique).\nenseigne(pierre, mathematiques).\nenseigne(pierre, informatique).\netudiant_de(E,P):-apprend(E,M), enseigne(P,M).\n"


- : token list =
[TPred "apprend"; TLPar; TPred "eve"; TAnd; TPred "mathematiques"; TRPar;
 TDot; TPred "apprend"; TLPar; TPred "benjamin"; TAnd; TPred "informatique";
 TRPar; TDot; TPred "apprend"; TLPar; TPred "benjamin"; TAnd;
 TPred "physique"; TRPar; TDot; TPred "enseigne"; TLPar; TPred "alice"; TAnd;
 TPred "physique"; TRPar; TDot; TPred "enseigne"; TLPar; TPred "pierre";
 TAnd; TPred "mathematiques"; TRPar; TDot; TPred "enseigne"; TLPar;
 TPred "pierre"; TAnd; TPred "informatique"; TRPar; TDot;
 TPred "etudiant_de"; TLPar; TVar "E"; TAnd; TVar "P"; TRPar; TIf;
 TPred "apprend"; TLPar; TVar "E"; TAnd; TVar "M"; TRPar; TAnd;
 TPred "enseigne"; TLPar; TVar "P"; TAnd; TVar "M"; TRPar; TDot]


In [25]:
extract_clauses (lexer program);;

extract_rightleft (lexer "etudiant_de(E,P):-apprend(E,M), enseigne(P,M)");;

extract_term_list_old (lexer "apprend(E,M), enseigne(P,M)");;

extract_term_list (lexer "apprend([A,B,C]), enseigne([B,C,A]), [a(A),b(B),c(C) | [] ]");;

inside_parenthesis (lexer "(E,M)");;

inside_parenthesis (lexer "[(E),[M],A]");;

tokenlist_to_term (lexer "apprend(benjamin, informatique)");;

tokenlist_to_term (lexer "[benjamin, informatique, [A,B,C | T ]]");;

let world1 = parser program;;

parser_term "apprend(benjamin, informatique)";;

parser_term_list "apprend(E,M), enseigne(P,M)";;

parser_clause "etudiant_de(E,P):-apprend(E,M), enseigne(P,M)";;

val program : string =
  "apprend(eve, mathematiques).\napprend(benjamin, informatique).\napprend(benjamin, physique).\nenseigne(alice, physique).\nenseigne(pierre, mathematiques).\nenseigne(pierre, informatique).\netudiant_de(E,P):-apprend(E,M), enseigne(P,M).\n"


- : token list =
[TAtom "apprend"; TLPar; TAtom "eve"; TAnd; TAtom "mathematiques"; TRPar;
 TDot; TAtom "apprend"; TLPar; TAtom "benjamin"; TAnd; TAtom "informatique";
 TRPar; TDot; TAtom "apprend"; TLPar; TAtom "benjamin"; TAnd;
 TAtom "physique"; TRPar; TDot; TAtom "enseigne"; TLPar; TAtom "alice"; TAnd;
 TAtom "physique"; TRPar; TDot; TAtom "enseigne"; TLPar; TAtom "pierre";
 TAnd; TAtom "mathematiques"; TRPar; TDot; TAtom "enseigne"; TLPar;
 TAtom "pierre"; TAnd; TAtom "informatique"; TRPar; TDot;
 TAtom "etudiant_de"; TLPar; TVar "E"; TAnd; TVar "P"; TRPar; TIf;
 TAtom "apprend"; TLPar; TVar "E"; TAnd; TVar "M"; TRPar; TAnd;
 TAtom "enseigne"; TLPar; TVar "P"; TAnd; TVar "M"; TRPar; TDot]


- : token list list =
[[TAtom "apprend"; TLPar; TAtom "eve"; TAnd; TAtom "mathematiques"; TRPar];
 [TAtom "apprend"; TLPar; TAtom "benjamin"; TAnd; TAtom "informatique";
  TRPar];
 [TAtom "apprend"; TLPar; TAtom "benjamin"; TAnd; TAtom "physique"; TRPar];
 [TAtom "enseigne"; TLPar; TAtom "alice"; TAnd; TAtom "physique"; TRPar];
 [TAtom "enseigne"; TLPar; TAtom "pierre"; TAnd; TAtom "mathematiques";
  TRPar];
 [TAtom "enseigne"; TLPar; TAtom "pierre"; TAnd; TAtom "informatique"; TRPar];
 [TAtom "etudiant_de"; TLPar; TVar "E"; TAnd; TVar "P"; TRPar; TIf;
  TAtom "apprend"; TLPar; TVar "E"; TAnd; TVar "M"; TRPar; TAnd;
  TAtom "enseigne"; TLPar; TVar "P"; TAnd; TVar "M"; TRPar]]


- : token list * token list =
([TAtom "etudiant_de"; TLPar; TVar "E"; TAnd; TVar "P"; TRPar],
 [TAtom "apprend"; TLPar; TVar "E"; TAnd; TVar "M"; TRPar; TAnd;
  TAtom "enseigne"; TLPar; TVar "P"; TAnd; TVar "M"; TRPar])


- : token list list =
[[TAtom "apprend"; TLPar; TVar "E"; TAnd; TVar "M"; TRPar];
 [TAtom "enseigne"; TLPar; TVar "P"; TAnd; TVar "M"; TRPar]]


...

- : term =
PList
 (PLCons
   ([Term ("benjamin", []); Term ("informatique", []);
     PList
      (PLCons
        ([Var (VarT ("A", 0)); Var (VarT ("B", 0)); Var (VarT ("C", 0))],
        Var (VarL ("T", 0))))],
   PList Empty))


val world1 : clause list =
  [Clause (Term ("apprend", [Term ("eve", []); Term ("mathematiques", [])]),
    []);
   Clause
    (Term ("apprend", [Term ("benjamin", []); Term ("informatique", [])]),
    []);
   Clause (Term ("apprend", [Term ("benjamin", []); Term ("physique", [])]),
    []);
   Clause (Term ("enseigne", [Term ("alice", []); Term ("physique", [])]),
    []);
   Clause
    (Term ("enseigne", [Term ("pierre", []); Term ("mathematiques", [])]),
    []);
   Clause
    (Term ("enseigne", [Term ("pierre", []); Term ("informatique", [])]), 
    []);
   Clause (Term ("etudiant_de", [Var (VarT ("E", 0)); Var (VarT ("P", 0))]),
    [Term ("apprend", [Var (VarT ("E", 0)); Var (VarT ("M", 0))]);
     Term ("enseigne", [Var (VarT ("P", 0)); Var (VarT ("M", 0))])])]


- : term =
Term ("apprend", [Term ("benjamin", []); Term ("informatique", [])])


- : term list =
[Term ("apprend", [Var (VarT ("E", 0)); Var (VarT ("M", 0))]);
 Term ("enseigne", [Var (VarT ("P", 0)); Var (VarT ("M", 0))])]


- : clause =
Clause (Term ("etudiant_de", [Var (VarT ("E", 0)); Var (VarT ("P", 0))]),
 [Term ("apprend", [Var (VarT ("E", 0)); Var (VarT ("M", 0))]);
  Term ("enseigne", [Var (VarT ("P", 0)); Var (VarT ("M", 0))])])


In [26]:
request world1 "etudiant_de(E, pierre)";;

There is : E = eve
There is : E = benjamin


- : unit = ()


In [27]:
request world1 "etudiant_de(E, pierre), etudiant_de(E, alice)";;

There is : E = benjamin


- : unit = ()


In [28]:
request world1 "etudiant_de(A, B)";;

There is : A = eve, B = pierre
There is : A = benjamin, B = pierre
There is : A = benjamin, B = alice


- : unit = ()


In [29]:
request world1 "etudiant_de(A, A)";;

This is false.


- : unit = ()


In [30]:
request world1 "apprend(A, A)";;

This is false.


- : unit = ()


In [31]:
request world1 "enseigne(A, A)";;

This is false.


- : unit = ()


In [32]:
request world1 "enseigne(alice, physique)";;

This is True.


- : unit = ()


In [33]:
request world1 "enseigne(alice, mathematiques)";; 

This is false.


- : unit = ()


## Le métro londonien

In [34]:
let program = "
connected(bond_street,oxford_circus,central).
connected(oxford_circus,tottenham_court_road,central).
connected(bond_street,green_park,jubilee).
connected(green_park,charing_cross,jubilee).
connected(green_park,piccadilly_circus,piccadilly).
connected(piccadilly_circus,leicester_square,piccadilly).
connected(green_park,oxford_circus,victoria).
connected(oxford_circus,piccadilly_circus,bakerloo).
connected(piccadilly_circus,charing_cross,bakerloo).
connected(tottenham_court_road,leicester_square,northern).
connected(leicester_square,charing_cross,northern).

nearby(X,Y):-connected(X,Y,L).
nearby(X,Y):-connected(X,Z,L),connected(Z,Y,L).

reachable(X,Y):-connected(X,Y,L).
reachable(X,Y):-connected(X,Z,L),reachable(Z,Y).

path(X,Y,noroute):-connected(X,Y,L).
path(X,Y,route(Z,R)):-connected(X,Z,L),path(Z,Y,R).";; (*connected(X,Y,L):-connected(Y,X,L).*)

val program : string =
  "\nconnected(bond_street,oxford_circus,central).\nconnected(oxford_circus,tottenham_court_road,central).\nconnected(bond_street,green_park,jubilee).\nconnected(green_park,charing_cross,jubilee).\nconnected(green_park,piccadilly_circus,piccadilly).\nconnected(piccadilly_circus,leicester_square,piccadilly)."... (* string length 816; truncated *)


In [35]:
let world = parser program;;

val world : clause list =
  [Clause
    (Term ("connected",
      [Term ("bond_street", []); Term ("oxford_circus", []);
       Term ("central", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("oxford_circus", []); Term ("tottenham_court_road", []);
       Term ("central", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("bond_street", []); Term ("green_park", []);
       Term ("jubilee", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("green_park", []); Term ("charing_cross", []);
       Term ("jubilee", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("green_park", []); Term ("piccadilly_circus", []);
       Term ("piccadilly", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("piccadilly_circus", []); Term ("leicester_square", []);
       Term ("piccadilly", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("green_park", []); Term ("oxford_circus", []);
       Term ("victoria", [])]),
    []);
   Cla

In [36]:
request world "connected(piccadilly_circus,leicester_square,piccadilly)";;

This is True.


- : unit = ()


In [37]:
request world "nearby(oxford_circus, charing_cross)";;

This is True.


- : unit = ()


In [38]:
request world "nearby(tottenham_court_road,W)";;

There is : W = leicester_square
There is : W = charing_cross


- : unit = ()


In [39]:
request world "reachable(bond_street, leicester_square)";;

This is True.
This is True.
This is True.
This is True.
This is True.


- : unit = ()


In [40]:
request world "connected(oxford_circus, bond_street, L)";;

This is false.


- : unit = ()


In [41]:
request world "path(oxford_circus,charing_cross,R)";;

There is : R = route(tottenham_court_road, route(leicester_square, noroute))
There is : R = route(piccadilly_circus, noroute)
There is : R = route(piccadilly_circus, route(leicester_square, noroute))


- : unit = ()


In [42]:
program;;

- : string =
"\nconnected(bond_street,oxford_circus,central).\nconnected(oxford_circus,tottenham_court_road,central).\nconnected(bond_street,green_park,jubilee).\nconnected(green_park,charing_cross,jubilee).\nconnected(green_park,piccadilly_circus,piccadilly).\nconnected(piccadilly_circus,leicester_square,piccadilly)."... (* string length 816; truncated *)
